In [15]:
import os
import tweepy
import pandas as pd

def get_tweets(hashtag):
    # Autenticación
    consumer_key = os.getenv("tw_consumer_key")
    consumer_secret = os.getenv("tw_consumer_secret")
    access_token = os.getenv("tw_access_token")
    access_token_secret = os.getenv("tw_access_token_secret")

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth)
    tweets_data = []
    for tweet in tweepy.Cursor(api.search_tweets, q=hashtag, lang="es").items(50):
        tweet_data = {
            'text': tweet.text,
            'user': tweet.user.screen_name,
            'is_retweet': hasattr(tweet, 'retweeted_status')
        }
        tweets_data.append(tweet_data)
    df = pd.DataFrame(tweets_data, columns=['text', 'user', 'is_retweet'])
    return df

In [16]:
# Set environment variables for testing (replace with your actual keys and tokens)
import os
os.environ["tw_consumer_key"] = "your_consumer_key"
os.environ["tw_consumer_secret"] = "your_consumer_secret"
os.environ["tw_access_token"] = "your_access_token"
os.environ["tw_access_token_secret"] = "your_access_token_secret"

# Mock the tweepy API for testing
from unittest.mock import patch

def mock_search_tweets(*args, **kwargs):
	class MockTweet:
		def __init__(self, text, user, is_retweet):
			self.text = text
			self.user = type('User', (object,), {'screen_name': user})()
			self.retweeted_status = is_retweet

	return iter([
		MockTweet("This is a test tweet #IA", "test_user1", False),
		MockTweet("Another test tweet #IA", "test_user2", True)
	])

with patch('tweepy.Cursor') as MockCursor:
	MockCursor.return_value.items = mock_search_tweets
	df = get_tweets("#IA")
	print(df)

                       text        user  is_retweet
0  This is a test tweet #IA  test_user1        True
1    Another test tweet #IA  test_user2        True


In [17]:
# Unit test for the function get_tweets

def test_get_tweets():
    # Use the mocked dataframe 'df' instead of calling the actual API
    assert df.shape[0] <= 50
    assert df.shape[1] == 3
    assert df.columns.tolist() == ['text', 'user', 'is_retweet']
    assert df['text'].dtype == 'object'
    assert df['user'].dtype == 'object'
    assert df['is_retweet'].dtype == 'bool'

# Run the test
test_get_tweets()